In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
from datetime import datetime

# === CONFIG ===
EXPORT_TO_EXCEL = True
OUTPUT_FILE = "reports/fx_major_heatmap.xlsx"
CURRENCY_LIST = ['USD','CAD', 'EUR', 'GBP', 'JPY', 'AUD', 'NZD', 'CHF','SGD','NOK']
TODAY_DATE = datetime.utcnow().strftime('%Y-%m-%d')

# === Initialize matrix ===
matrix = pd.DataFrame(index=CURRENCY_LIST, columns=CURRENCY_LIST, dtype=float)

def get_daily_pct_change(ticker):
    try:
        data = yf.download(ticker, period="2d", interval="1d", progress=False)
        if len(data) < 2:
            return None
        open_val = data['Open'].iloc[-1].item()
        close_val = data['Close'].iloc[-1].item()
        return (close_val - open_val) / open_val * 100
    except Exception as e:
        print(f"[⚠️] Error fetching {ticker}: {e}")
        return None

# === Build matrix ===
for base in CURRENCY_LIST:
    for quote in CURRENCY_LIST:
        if base == quote:
            matrix.at[base, quote] = np.nan
            continue
        pair = f"{base}{quote}=X"
        pct_change = get_daily_pct_change(pair)
        if pct_change is not None:
            matrix.at[base, quote] = round(pct_change, 2)

# === Display matrix ===
print(f"[📊] FX % Change Heatmap for {TODAY_DATE}")
print(matrix)

# === Export to Excel (optional) ===
if EXPORT_TO_EXCEL:
    os.makedirs("reports", exist_ok=True)
    with pd.ExcelWriter(OUTPUT_FILE, engine='xlsxwriter') as writer:
        matrix.to_excel(writer, sheet_name='Heatmap')
        workbook  = writer.book
        worksheet = writer.sheets['Heatmap']
        fmt = workbook.add_format({'num_format': '0.00', 'align': 'center'})

        # Apply conditional formatting
        worksheet.conditional_format('B2:I9', {
            'type': '3_color_scale',
            'min_color': "#63BE7B",   # green
            'mid_color': "#FFEB84",   # yellow
            'max_color': "#F8696B",   # red
        })
    print(f"[💾] Exported FX heatmap to: {OUTPUT_FILE}")



1 Failed download:
['SGDCAD=X']: HTTPError('HTTP Error 404: ')


[📊] FX % Change Heatmap for 2025-07-04
      USD   CAD   EUR   GBP   JPY   AUD   NZD   CHF   SGD   NOK
USD   NaN   NaN -0.19  0.00 -0.31  0.24  0.22   NaN -0.07  0.15
CAD -0.11   NaN -0.30 -0.13 -0.40  0.17  0.10   NaN -0.22  0.03
EUR  0.19  0.31   NaN  0.18   NaN   NaN  0.40   NaN  0.13   NaN
GBP  0.00   NaN -0.18   NaN -0.31   NaN   NaN   NaN   NaN  0.08
JPY  0.30  0.43  0.13  0.32   NaN  0.61  0.52  0.26   NaN  0.94
AUD -0.24   NaN -0.44 -0.22   NaN   NaN   NaN   NaN   NaN -0.18
NZD -0.22   NaN -0.53 -0.23 -0.53  0.09   NaN   NaN -0.28   NaN
CHF   NaN  0.36  0.03  0.20   NaN  0.50  0.43   NaN  0.48  0.32
SGD  0.07   NaN -0.07  0.11 -0.09  0.40  0.33  0.14   NaN  0.20
NOK   NaN   NaN -0.31 -0.10 -0.34   NaN   NaN  0.06   NaN   NaN
[💾] Exported FX heatmap to: reports/fx_major_heatmap.xlsx
